<a href="https://colab.research.google.com/github/bjungweapon/mjc.ai.ml/blob/BDU/bdu.9.3.%EC%9D%80%ED%96%89%EA%B3%A0%EA%B0%9D%EB%B6%84%EB%A5%98.KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 문제 정의
   ( 대출 여부를 결정하는 분류 문제 )

   . 은행에서 수립한 고객 5000 명의 금융 정보에 있는 대출 여부가 들어 잇는 참조 데이터 이용.
  . 대출 여부를 분류하는 가장 최적의 이웃의 크기값을 구하고 분류 정확도를 산출

  . 7:3의 비율로 트레이닝.

  . z 값을 이용하여 정규화 한다.

   ---> 이때 사용하는 알고리즘은 어떤 것을 쓸까?
       ( KNN 알고리즘을 사용하자. 분류 해야 하는 것)

       

 데이터 선택.
   Personal Loan 은 캐글에서 제고하는 5,000 명의 은행 고객의 12가지 속성을 기록한 데이타 셋 활용.

   KNN과 관련이 적은 속성 (ID, ZIP Code )등을 제거한 수정 데이터 셋 사용


   종속 변수 :   Personal Loadn (대출 여부 ) : 1 (대출), 0 ( 거부) 를 분류.
   